In [11]:
import torchvision as thv
import numpy as np
import os

In [8]:
# Doanload Data
if not os.path.isdir("./MNIST"):
    train = thv.datasets.MNIST('./', download=True, train=True)
    val = thv.datasets.MNIST('./', download=True, train=False)

    print(train.data.shape, len(train.targets))

In [608]:
class layer_t:
    def backward_check(self, dh_l1, i=7, j=432):
        # PS: since the matrix dw is really sparse, it's good to try and visualize it
        # for picking i,j values or try many different values
        i, j = 7, 432

        k = np.argwhere(dh_l1 == 1)[0, 1]
        epsilon_ij = np.random.normal(0, 1)
        epsilon = np.zeros_like(self.w)
        epsilon[i, j] = epsilon_ij

        dw_ij_num = np.matmul((self.w + epsilon), self.hl.T)[k, 0] - np.matmul((self.w - epsilon), self.hl.T)[k, 0]
        dw_ij_den = 2 * epsilon_ij

        dw_ij = dw_ij_num / dw_ij_den

        return dw_ij, self.dw[i,j]
    
    def backward_check_h(self, dh_l1, i=7):
        # PS: since the matrix hl is really sparse, it's good to try and visualize it
        # for picking i values or try many different values
        k = np.argwhere(dh_l1 == 1)[0, 1]
        
        epsilon_i = np.random.normal(0, 1)
        epsilon = np.zeros_like(self.hl)
        epsilon[0, i] = epsilon_i

        dw_i_num = np.matmul((self.hl + epsilon), self.hl.T) - np.matmul((self.hl - epsilon), self.hl.T)
        dw_i_den = 2 * epsilon_i

        dw_i = dw_i_num / dw_i_den
        
        return dw_i, self.hl[0, k]
    
    def backward_check_b(self, dh_l1, i=7):
        # PS: since the matrix hl is really sparse, it's good to try and visualize it
        # for picking i values or try many different valuesgo
        k = np.argwhere(dh_l1 == 1)[0, 1]
        
        epsilon_i = np.random.normal(0, 1)
        epsilon = np.zeros_like(self.b)
        epsilon[i] = epsilon_i

        dw_i_num = np.matmul((self.b + epsilon), self.hl)[k,0] - np.matmul((self.b - epsilon), self.hl)[k,0]
        dw_i_den = 2 * epsilon_i

        dw_i = dw_i_num / dw_i_den
        
        return dw_i, self.hl[0, k]
    
        
    def zero_grad(self):
        pass

In [ ]:
class linear_t(layer_t):
    def __init__(self):
        # input size
        self.A = 784 
        
        # mini batch size
        self.BATCH = 1
        
        # class count
        self.C = 10
        
        # Fix seed
        np.random.seed(546)
        
        # Define normalized w, and b
        self.w = np.random.normal(loc=0, scale=1, size=(self.C, self.A))
        self.w = self.w / np.linalg.norm(self.w)

        self.b = np.random.normal(loc=0, scale=1, size=(self.C, 1))
        self.b = self.b / np.linalg.norm(self.b)
        
        # Define placeholder gradients to be filled in backward step
        self.dw = np.zeros_like(self.w)
        self.db = np.zeros_like(self.b)
        
        

    def forward(self, h_l):
        h_l1 = np.matmul(h_l, np.transpose(self.w)) + self.b.T

        # cache hˆl in forward because we will need it to compute
        # dw in backward
        self.hl = h_l
        
        return h_l1


    def backward(self, dh_l1):
        dh_l = np.matmul(dh_l1, self.w)
        dw = np.matmul(dh_l1.T, self.hl)
        db = np.matmul(dh_l1.T, np.ones([self.hl.shape[0], 1]))

        self.dw, self.db = dw, db

        # notice that there is no need to cache dh_{l+1}
        return dh_l
    
    def zero_grad(self):
        # useful to delete the stored backprop gradients of the
        # previous mini-batch before you start a new mini-batch
        self.dw, self.db = 0 * self.dw, 0 * self.db

In [651]:
class relu_t(layer_t):
    def forward(self, h_l):
        return np.maximum(0, h_l)
    
    def backward(self, dh_l1):
        x = np.array(dh_l1, copy=True)
        x[x <= 0] = 0
        x[x > 0] = 1
        return x

In [639]:
class softmax_cross_entropy_t(layer_t):
    def __init__(self):
        # no parameters, nothing to initialize
        self.h_l1 = None
        self.y = None

    def forward(self, h_l, y):
        expd = np.exp(h_l)
        h_l1 = expd / np.sum(expd, axis=1,keepdims=True) 
        
        self.h_l1 = h_l1
        self.y = y
       
        # compute average loss ell(y) over a mini-batch
        BATCH_SIZE = h_l1.shape[0]
        ell = np.sum(-np.log(h_l1[range(BATCH_SIZE), self.y])) / BATCH_SIZE
        
        # compute the error of predictions
        error = np.sum(np.not_equal(self.y, np.argmax(h_l1, axis=1)))/BATCH_SIZE
        
        return ell, error

    def backward(self):
        # as we saw in the notes, the backprop input to the
        # loss layer is 1, so this function does not take any
        # arguments
        
        B = self.y.shape[0] # batch size
        self.h_l1[range(B),self.y] -=1
        
        return self.h_l1/B

In [210]:
h_l = np.array([[0, 1, 2], # mini-batch1
                [0, 5, 9], # mini-batch2
                [1, 3, 2],
                [1, 3, 2]])# mini-batch3

y_i = [0, 3, 4]

                
print(np.argmax(h_l, axis=1))

# print(np.sum(W, axis=1))
# np.sum(W, axis=1).shape

[2 2 1 1]


In [352]:
def subsample(dataset, num_samples, num_classes):
    # Get subsample by sampling
    num_samples_per_class = int(num_samples/num_classes)
    dataset = np.array(list(map(lambda elt: (np.array(elt[0]).flatten(),elt[1]), dataset)))
    X, y = map(lambda x: np.array(list(x)),zip(*dataset))
    subsamplesX, subsamplesY = None, None

    for c in range(num_classes):
        # Get all X's where label == c
        mask = np.argwhere(y == c)
        class_X, class_Y = X[mask], y[mask]
        class_X = class_X.reshape((class_X.shape[0], X.shape[1]))
        
        # Get all X's where label == c, inds = index array
        inds = np.random.randint(0, high=class_Y.shape[0], size=num_samples_per_class)
        subsamplesX = class_X[inds] if subsamplesX is None else np.concatenate([subsamplesX, class_X[inds]])
        
        # Convert Y to one-hot
#         class_subsample = np.zeros((num_samples_per_class, num_classes))
#         class_subsample[np.arange(num_samples_per_class), class_Y[inds]] = 1
#         subsamplesY = class_subsample if subsamplesY is None else np.concatenate([subsamplesY,class_subsample])

        # Get all Y's where label == c
        subsamplesY = class_Y[inds] if subsamplesY is None else np.concatenate([subsamplesY, class_Y[inds]])
    
    # shuffle the arrays
    A = subsamplesX.shape[1]
    B = subsamplesX.shape[0]
    shuf_inds = np.random.shuffle(np.array(range(B)))
    return subsamplesX[shuf_inds].reshape((B,A)), subsamplesY[shuf_inds].reshape((num_samples,))

In [650]:
# def one_hot(y_i):
#     hot_y = np.zeros((10,)) # C = 10
#     hot_y[y_i] = 1
#     return hot_y

In [213]:
trainX, trainY = subsample(dataset=train, num_samples=30000, num_classes=10)
valX, valY = subsample(dataset=val, num_samples=5000, num_classes=10)

Estimated gradient = -0.0, true gradient = 0.0


In [620]:
######## CODE FOR CHECKING GRADIENT MATCHING #############
l1 = linear_t()
l2 = relu_t()
l3 = softmax_cross_entropy_t()

h1 = l1.forward(np.array([trainX[i]]))
h2 = l2.forward(h1)
ell, error = l3.forward(h2, np.array([trainY[i]]))


dh2 = l3.backward()
dh1 = l2.backward(dh2)
dx = l1.backward(dh1)

estimate_grad, true_grad = l1.backward_check(dh1)

print('Estimated gradient = {}, true gradient = {}'.format(estimate_grad, true_grad))
            
#         estimate_grad, true_grad = NN.backward_check_h(dhl1, i=j)
#         if abs(estimate_grad-true_grad) > 0.003:
#             print('Estimated h_l1 = {}, true h_l1 = {}'.format(estimate_grad, true_grad))
            
#         estimate_grad, true_grad = NN.backward_check_b(dhl1, i=int(j*10/784))
#         if abs(estimate_grad-true_grad) > 0.003:
#             print('Estimated b = {}, true b = {}'.format(estimate_grad, true_grad))

h^{l+1}_y: [0.00860261] [[1.00863972 1.03331488 1.00863972 1.09213666 1.01447528 1.03662148
  1.06195386 1.08084117 1.00863972 1.9446767 ]]
Estimated gradient = -0.0, true gradient = 254.0


In [353]:
trainX, trainY = subsample(dataset=train, num_samples=30000, num_classes=10)

print(trainX.shape, trainY.shape)

(30000, 784) (30000,)


In [652]:
##################################################################
##################################################################
############### TRAINING THE NEURAL NETWORK ######################
##################################################################
##################################################################

l1, l2, l3 = linear_t(), relu_t(), softmax_cross_entropy_t()
net = [l1, l2, l3]

iters = 10000
lr = 0.01

batch_size = 1
training_data_size = trainX.shape[0]

# train for at least 1000 iterations
for t in range(iters):
    # 1. sample a mini-batch of size bb = 32
    # each image in the mini-batch is chosen uniformly randomly from the # training dataset
    inds = np.random.randint(0,high=training_data_size, size=batch_size)
    x, y = trainX[inds], trainY[inds]
#     print(y)
    
    # 2. zero gradient buffer
    for l in net: 
        l.zero_grad()
        
    # 3. forward pass
    h1 = l1.forward(x)
    h2 = l2.forward(h1)
    ell, error = l3.forward(h2, y)
    
    print("class predictions: ", l3.h_l1)
    # 4. Backward pass
    dh2 = l3.backward()
    dh1 = l2.backward(dh2)
    dx  = l1.backward(dh1)
    
    # 5. gather backprop gradients
    dw, db = l1.dw, l1.db
    
    # 6. print some quantities for logging and debugging
    print(f"Iteration #{t} has ell:{ell} and error: {error}")
#     print(t, np.linalg.norm(dw/l1.w), np.linalg.norm(db/l1.b))
    
    # 7. one step of SGD
    l1.w = l1.w - lr*dw
    l1.b = l1.b - lr*db

class predictions:  [[0.01777996 0.09647651 0.00374286 0.72394944 0.00374286 0.00374286
  0.09886172 0.00374286 0.01075427 0.03720666]]
Iteration #0 has ell:5.58790516157247 and error: 1.0
class predictions:  [[0.08467492 0.08467492 0.08467492 0.08467492 0.08467492 0.08467492
  0.08467492 0.23792575 0.08467492 0.08467492]]
Iteration #1 has ell:2.4689358618405257 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #2 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #3 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #4 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #5 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #6 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1

Iteration #142 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #143 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #144 has ell:2.3025850929940455 and error: 0.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #145 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #146 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #147 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #148 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #149 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #150 has ell:2.3025850

Iteration #290 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #291 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #292 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #293 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #294 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #295 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #296 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #297 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #298 has ell:2.3025850

Iteration #412 has ell:2.3025850929940455 and error: 0.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #413 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #414 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #415 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #416 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #417 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #418 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #419 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #420 has ell:2.3025850

class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #612 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #613 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #614 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #615 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #616 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #617 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #618 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #619 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.

...

Iteration #9069 has ell:2.3025850929940455 and error: 0.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9070 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9071 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9072 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9073 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9074 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9075 has ell:2.3025850929940455 and error: 0.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9076 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9077 has ell:

Iteration #9305 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9306 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9307 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9308 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9309 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9310 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9311 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9312 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9313 has ell:

class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9498 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9499 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9500 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9501 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9502 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9503 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9504 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9505 has ell:2.3025850929940455 and error: 0.0
class predictions:  [[0.

Iteration #9685 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9686 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9687 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9688 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9689 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9690 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9691 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9692 has ell:2.3025850929940455 and error: 0.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9693 has ell:

Iteration #9848 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9849 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9850 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9851 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9852 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9853 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9854 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9855 has ell:2.3025850929940455 and error: 1.0
class predictions:  [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Iteration #9856 has ell: